In [1]:
import hddm # 导入hddm包
from patsy import dmatrix # 从patsy包中导入dmatrix包
import warnings # 导入warnings包

# 用来组织弹出警告提示
warnings.filterwarnings('ignore', category=FutureWarning)

# 导入数据，使用hddm中的load_csv函数
data = hddm.load_csv(r'D:\python_data_file\TFA_HDDM\combined_data.csv', encoding='utf-8')
data

ModuleNotFoundError: No module named 'hddm'

In [ ]:
# 数据准备
# 从数据中提取四个数据集，用于拟合模型
# 全数据集
all_data = data
# 只包含刺激2和刺激3的trial数据集（安全且确定组）
safe_certain_df = data[(data['similarity']==2)|(data['similarity']==3)]
# 只包含刺激5和刺激6的trial数据集（不确定组）
uncertain_df = data[(data['similarity']==5)|(data['similarity']==6)]
# 只包含刺激8和9的trial数据集（威胁且确定组）
threat_certain_df = data[(data['similarity']==8)|(data['similarity']==9)]

# 名字与其对应的数据集组成的字典
data_dic = {'all_df':all_data,'safe_certain_df':safe_certain_df,'uncertain_df':uncertain_df,'threat_certain_df':threat_certain_df}

#### 第一个问题

In [ ]:
# mPFC的能量活动是否与决策阈限变化显著相关，这种相关性在不同确定性刺激条件下是否有差异？
# 对于数据列表中的每一个数据集

# 设定影响a的变量，这里选取FPZ每个trial上的alpha波能量为例子，当然也可以选取不同的能量，或者对多个通道做平均值，然后再分析
var = 'alpha_FPZ'

for name,df in data_dic.items():
    # 声明模型
    model = hddm.HDDMRegressor(df, f"a~{var}", bias=True, p_outlier=0.05, keep_regressor_trace=True, group_only_regressors=True)
    # 找MCMC起始点
    model.find_starting_values()
    # 使用MCMC抽样估计参数值，抽5000，前2000样本算作马尔科夫链启动时预热样本，需要丢弃，将MCMC的采样过程信息保存为trace文件，形式是二进制文本‘pickle’
    model.sample(5000, burn=2000, dbname=rf'D:\python_data_file\TFA_HDDM\trace_mono_{name}', db='pickle')
    # 保存模型拟合的结果（比如参数的值等等），路径为预先设定好的路径，名字为model_mono...
    model.save(rf'D:\python_data_file\TFA_HDDM\model_mono_{name}')


In [1]:
# 把上面四个模型结果加载，以便提取模型中的参数值
m1 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_mono_all_df')
m2 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_mono_safe_certain_df')
m3 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_mono_uncertain_df')
m4 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_mono_threat_certain_df')

# 提取这四个模型中自变量的系数后验概率密度图（a~β*power）
m1_beta = m1.nodes_db.node[f'a_{var}']
m2_beta = m2.nodes_db.node[f'a_{var}']
m3_beta = m3.nodes_db.node[f'a_{var}']
m4_beta = m4.nodes_db.node[f'a_{var}']

# 画这四个系数的后验概率密度图
hddm.analyze.plot_posterior_nodes([m1_beta, m2_beta, m3_beta,m4_beta])

# 贝叶斯统计检验
print((m4_beta.trace()>0).mean())

NameError: name 'hddm' is not defined

#### 第二个问题

In [ ]:
# 设定影响a的变量，这里选取FPZ每个trial上的alpha波能量为例子，加入状态焦虑探索人格是否有调节作用
var1 = 'alpha_FPZ'
var2= 'Sanxiety'

for name,df in data_dic.items():
    # 声明模型
    model = hddm.HDDMRegressor(df, f"a~{var1}*{var2}", bias=True, p_outlier=0.05, keep_regressor_trace=True, group_only_regressors=True)
    # 找MCMC起始点
    model.find_starting_values()
    # MCMC抽样，抽5000，前2000样本算作马尔科夫链启动时预热样本，需要丢弃，将MCMC的采样过程信息保存为trace文件，形式是二进制文本‘pickle’
    model.sample(5000, burn=2000, dbname=rf'D:\python_data_file\TFA_HDDM\trace_bin_{name}', db='pickle')
    # 保存模型拟合的结果（比如参数的值等等），路径为预先设定好的路径，名字为model_bin...
    model.save(rf'D:\python_data_file\TFA_HDDM\model_bin_{name}')

    # 把上面四个模型结果加载，以便提取模型中的参数值
m1 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_bin_all_df')
m2 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_bin_safe_certain_df')
m3 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_bin_uncertain_df')
m4 = hddm.load(r'D:\python_data_file\TFA_HDDM\model_bin_threat_certain_df')

# 提取这四个模型中交互项的系数后验概率密度图（a~β*power）
m1_beta = m1.nodes_db.node[f'a_{var1}:{var2}']
m2_beta = m2.nodes_db.node[f'a_{var1}:{var2}']
m3_beta = m3.nodes_db.node[f'a_{var1}:{var2}']
m4_beta = m4.nodes_db.node[f'a_{var1}:{var2}']

# 画这四个系数的后验概率密度图
hddm.analyze.plot_posterior_nodes([m1_beta, m2_beta, m3_beta,m4_beta])

# 贝叶斯统计检验
# 略